# Compare models

In [ ]:
import logging
import random
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

pd.options.display.max_rows = 120
pd.options.display.min_rows = 50

import vaep
import vaep.imputation
from vaep import sampling
from vaep.io import datasplits
from vaep.analyzers import compare_predictions


import vaep.nb
matplotlib.rcParams['figure.figsize'] = [10.0, 8.0]


logging.basicConfig(level=logging.INFO)

In [ ]:
models = ['collab', 'DAE', 'VAE']

In [ ]:
# files and folders
folder_experiment:str = 'runs/experiment_03/df_intensities_proteinGroups_long_2017_2018_2019_2020_N05015_M04547/Q_Exactive_HF_X_Orbitrap_Exactive_Series_slot_#6070' # Datasplit folder with data for experiment
folder_data:str = '' # specify data directory if needed
file_format: str = 'pkl' # change default to pickled files
fn_rawfile_metadata: str = 'data/files_selected_metadata.csv' # Machine parsed metadata from rawfile workflow

In [ ]:
args = vaep.nb.Config()

args.fn_rawfile_metadata = fn_rawfile_metadata
del fn_rawfile_metadata

args.folder_experiment = Path(folder_experiment)
del folder_experiment
args.folder_experiment.mkdir(exist_ok=True, parents=True)

args.file_format = file_format
del file_format

args = vaep.nb.add_default_paths(args, folder_data=folder_data)
del folder_data

args

In [ ]:
data = datasplits.DataSplits.from_folder(args.data, file_format=args.file_format) 

In [ ]:
fig, axes = plt.subplots(1, 2, sharey=True, figsize=(18,10))

_ = data.val_y.unstack().notna().sum(axis=1).sort_values().plot(
        rot=90,
        ax=axes[0],
        title='Validation data',
        ylabel='number of feat')
_ = data.test_y.unstack().notna().sum(axis=1).sort_values().plot(
        rot=90,
        ax=axes[1],
        title='Test data')

## Across data completeness

In [ ]:
freq_feat = sampling.frequency_by_index(data.train_X, 0)
freq_feat.name = 'freq'
freq_feat.head() # training data

In [ ]:
prop = freq_feat / len(data.train_X.index.levels[0])
prop.to_frame()

## reference methods

- drawing from shifted normal distribution
- drawing from (-) normal distribution?
- median imputation

In [ ]:
data.to_wide_format()
data.train_X

In [ ]:
mean = data.train_X.mean()
std = data.train_X.std()

imputed_shifted_normal = vaep.imputation.impute_shifted_normal(data.train_X)
imputed_shifted_normal

In [ ]:
imputed_normal = vaep.imputation.impute_shifted_normal(data.train_X, mean_shift=0.0, std_shrinkage=1.0)
imputed_normal

In [ ]:
medians_train = data.train_X.median()
medians_train.name = 'median'

## load predictions

In [ ]:
split='test'
pred_files =  [f for f in args.out_preds.iterdir() if split in f.name]
pred_test = compare_predictions.load_predictions(pred_files)
# pred_test = pred_test.join(medians_train, on=prop.index.name)
pred_test['random shifted normal'] = imputed_shifted_normal
pred_test['random normal'] = imputed_normal
pred_test = pred_test.join(freq_feat, on=freq_feat.index.name)
pred_test

In [ ]:
feature_names = pred_test.index.levels[-1]
M = len(feature_names)
pred_test.loc[pd.IndexSlice[:, feature_names[random.randint(0, M)]], :]

In [ ]:
options = random.sample(set(freq_feat.index), 1)
pred_test.loc[pd.IndexSlice[:, options[0]], :]

In [ ]:
split='val'
pred_files =  [f for f in args.out_preds.iterdir() if split in f.name]
pred_val = compare_predictions.load_predictions(pred_files)
# pred_val = pred_val.join(medians_train, on=freq_feat.index.name)
pred_val['random shifted normal'] = imputed_shifted_normal
# pred_val = pred_val.join(freq_feat, on=freq_feat.index.name)

# 
errors_val = pred_val.drop('observed', axis=1).sub(pred_val['observed'], axis=0)
errors_val.describe() # over all samples, and all features

Describe absolute error

In [ ]:
errors_val.abs().describe() # over all samples, and all features

In [ ]:
c_error_min = 4.5
mask = (errors_val[models].abs() > c_error_min).any(axis=1)
errors_val.loc[mask].sort_index(level=1)

In [ ]:
errors_val = errors_val.abs().groupby(freq_feat.index.name).mean() # absolute error
errors_val = errors_val.join(freq_feat)
errors_val = errors_val.sort_values(by=freq_feat.name, ascending=True)
errors_val

Some interpolated features are missing

In [ ]:
errors_val.describe()  # mean of means

In [ ]:
c_avg_error = 2
mask = (errors_val[models] >= c_avg_error).any(axis=1)
errors_val.loc[mask]

In [ ]:
errors_val_smoothed = errors_val.copy()
errors_val_smoothed[errors_val.columns[:-1]] = errors_val[errors_val.columns[:-1]].rolling(window=200, min_periods=1).mean()
ax = errors_val_smoothed.plot(x=freq_feat.name, ylabel='rolling error average', ylim=(0,2))

In [ ]:
errors_val_smoothed.describe()

In [ ]:
vaep.savefig(
    ax.get_figure(),
    folder=args.out_figures,
    name='performance_methods_by_completness')

### Average errors
- see how smoothing is done, here `collab`
- shows how features are distributed in training data

In [ ]:
# scatter plots to see spread
model = models[0]
ax = errors_val.plot.scatter(x=prop.name, y=model, c='darkblue', ylim=(0,2))

- [ ] plotly plot with number of observations the mean for each feature is based on
- [ ] 